# Import libraries

In [ ]:
import random, os

from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import f1_score
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,GradientBoostingClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import copy
%matplotlib inline

# Configs

In [ ]:
BASE_PATH = './'

SEED = 42
n_est = 10000
n_class = 3

def seed_everything(seed: int):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)

seed_everything(SEED)

# EDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/경진대회/train.csv', index_col=['index'])
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/경진대회/test.csv', index_col=['index'])
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/경진대회/sample_submission.csv')

In [ ]:
.train.head()

In [ ]:
train.shape

In [ ]:
train['occyp_type'].unique()

In [ ]:
train['occyp_type'].value_counts()

In [ ]:
train['occyp_type'].value_counts()

In [ ]:
test.head()

In [ ]:
# 결측치 처리
train['occyp_type'] = train['occyp_type'].fillna('Laborers')
test['occyp_type'] = test['occyp_type'].fillna('Laborers')

### one hot encording

In [ ]:
# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
train.head()

In [ ]:
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test

In [ ]:
print(train_x.shape, train_y.shape, test.shape)

# Training

In [ ]:
train_prob_dict = {}
test_prob_dict = {}

In [ ]:
n_fold = 20

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': n_est,
    'objective': 'multiclass',
    'random_state': SEED,
    'learning_rate': 0.005,
    'reg_alpha': 1.56e-05,
    'reg_lambda': 0.03167,
    'num_leaves': 255,
    'num_class': n_class,
    'max_depth': 99,
    'max_bin': 454,
    'subsample':0.8,
    'subsamle_freq':3,
    'min_child_samples':2,
    'colsample_bytree':0.39
}

lgb_pred = np.zeros([len(test),3])
train_proba = np.zeros((train_x.shape[0], n_class), dtype=float)

kfold = KFold(n_splits=n_fold, random_state=SEED, shuffle=True)


for fold, (train_idx, val_idx) in enumerate(kfold.split(train_x)):
  print(f"===== {fold} time =====")

  X_train, X_val = train_x.iloc[train_idx, :], train_x.iloc[val_idx, :]
  y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

  clf = LGBMClassifier(**lgb_params)
  clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=100)

  lgb_pred += clf.predict_proba(test_x)/ n_fold
  train_proba[val_idx] = clf.predict_proba(X_val)

train_prob_dict['lgb1'] = train_proba
test_prob_dict['lgb1'] = lgb_pred

===== 0 time =====
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.848771
[200]	valid_0's multi_logloss: 0.821456
[300]	valid_0's multi_logloss: 0.801003
[400]	valid_0's multi_logloss: 0.784895
[500]	valid_0's multi_logloss: 0.772209
[600]	valid_0's multi_logloss: 0.762355
[700]	valid_0's multi_logloss: 0.754894
[800]	valid_0's multi_logloss: 0.748178
[900]	valid_0's multi_logloss: 0.742292
[1000]	valid_0's multi_logloss: 0.737736
[1100]	valid_0's multi_logloss: 0.733547
[1200]	valid_0's multi_logloss: 0.729441
[1300]	valid_0's multi_logloss: 0.726744
[1400]	valid_0's multi_logloss: 0.724365
[1500]	valid_0's multi_logloss: 0.722235
[1600]	valid_0's multi_logloss: 0.720374
[1700]	valid_0's multi_logloss: 0.719063
[1800]	valid_0's multi_logloss: 0.718057
[1900]	valid_0's multi_logloss: 0.717239
[2000]	valid_0's multi_logloss: 0.71648
[2100]	valid_0's multi_logloss: 0.715653
[2200]	valid_0's multi_logloss: 0.714973
[2300]	valid_0's multi_logloss: 0.714681
[2400]	valid_0's multi_logloss: 0.714561
Early stopping, best itera

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.814353
[200]	valid_0's multi_logloss: 0.787947
[300]	valid_0's multi_logloss: 0.768352
[400]	valid_0's multi_logloss: 0.753317
[500]	valid_0's multi_logloss: 0.74119
[600]	valid_0's multi_logloss: 0.731213
[700]	valid_0's multi_logloss: 0.723952
[800]	valid_0's multi_logloss: 0.717527
[900]	valid_0's multi_logloss: 0.71198
[1000]	valid_0's multi_logloss: 0.707228
[1100]	valid_0's multi_logloss: 0.703289
[1200]	valid_0's multi_logloss: 0.699408
[1300]	valid_0's multi_logloss: 0.696454
[1400]	valid_0's multi_logloss: 0.694183
[1500]	valid_0's multi_logloss: 0.691792
[1600]	valid_0's multi_logloss: 0.689685
[1700]	valid_0's multi_logloss: 0.688106
[1800]	valid_0's multi_logloss: 0.686629
[1900]	valid_0's multi_logloss: 0.685707
[2000]	valid_0's multi_logloss: 0.684872
[2100]	valid_0's multi_logloss: 0.684113
[2200]	valid_0's multi_logloss: 0.68363
[2300]	valid_0's multi_logloss: 0.683018
[2400]	valid_0's multi_logloss: 0.682722
[2500]	valid_0's multi_loglo

...

[100]	valid_0's multi_logloss: 0.851146
[200]	valid_0's multi_logloss: 0.819998
[300]	valid_0's multi_logloss: 0.798098
[400]	valid_0's multi_logloss: 0.780502
[500]	valid_0's multi_logloss: 0.766481
[600]	valid_0's multi_logloss: 0.755537
[700]	valid_0's multi_logloss: 0.747196
[800]	valid_0's multi_logloss: 0.74012
[900]	valid_0's multi_logloss: 0.734334
[1000]	valid_0's multi_logloss: 0.729525
[1100]	valid_0's multi_logloss: 0.724963
[1200]	valid_0's multi_logloss: 0.72118
[1300]	valid_0's multi_logloss: 0.718333
[1400]	valid_0's multi_logloss: 0.715947
[1500]	valid_0's multi_logloss: 0.713826
[1600]	valid_0's multi_logloss: 0.711896
[1700]	valid_0's multi_logloss: 0.709964
[1800]	valid_0's multi_logloss: 0.70883
[1900]	valid_0's multi_logloss: 0.707885
[2000]	valid_0's multi_logloss: 0.707023
[2100]	valid_0's multi_logloss: 0.706471
[2200]	valid_0's multi_logloss: 0.706082
[2300]	valid_0's multi_logloss: 0.705791
[2400]	valid_0's multi_logloss: 0.705625
[2500]	valid_0's multi_loglo

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.823893
[200]	valid_0's multi_logloss: 0.793667
[300]	valid_0's multi_logloss: 0.770967
[400]	valid_0's multi_logloss: 0.752908
[500]	valid_0's multi_logloss: 0.73782
[600]	valid_0's multi_logloss: 0.725932
[700]	valid_0's multi_logloss: 0.716524
[800]	valid_0's multi_logloss: 0.708297
[900]	valid_0's multi_logloss: 0.701242
[1000]	valid_0's multi_logloss: 0.695227
[1100]	valid_0's multi_logloss: 0.68982
[1200]	valid_0's multi_logloss: 0.685503
[1300]	valid_0's multi_logloss: 0.681485
[1400]	valid_0's multi_logloss: 0.678179
[1500]	valid_0's multi_logloss: 0.674956
[1600]	valid_0's multi_logloss: 0.67213
[1700]	valid_0's multi_logloss: 0.669804
[1800]	valid_0's multi_logloss: 0.66805
[1900]	valid_0's multi_logloss: 0.666148
[2000]	valid_0's multi_logloss: 0.664618
[2100]	valid_0's multi_logloss: 0.663501
[2200]	valid_0's multi_logloss: 0.662048
[2300]	valid_0's multi_logloss: 0.660761
[2400]	valid_0's multi_logloss: 0.659734
[2500]	valid_0's multi_loglos

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.852543
[200]	valid_0's multi_logloss: 0.823687
[300]	valid_0's multi_logloss: 0.802088
[400]	valid_0's multi_logloss: 0.785207
[500]	valid_0's multi_logloss: 0.771915
[600]	valid_0's multi_logloss: 0.761704
[700]	valid_0's multi_logloss: 0.753778
[800]	valid_0's multi_logloss: 0.747252
[900]	valid_0's multi_logloss: 0.741501
[1000]	valid_0's multi_logloss: 0.736801
[1100]	valid_0's multi_logloss: 0.732857
[1200]	valid_0's multi_logloss: 0.729602
[1300]	valid_0's multi_logloss: 0.727085
[1400]	valid_0's multi_logloss: 0.724791
[1500]	valid_0's multi_logloss: 0.722824
[1600]	valid_0's multi_logloss: 0.721085
[1700]	valid_0's multi_logloss: 0.719843
[1800]	valid_0's multi_logloss: 0.718541
[1900]	valid_0's multi_logloss: 0.7175
[2000]	valid_0's multi_logloss: 0.716777
[2100]	valid_0's multi_logloss: 0.716176
[2200]	valid_0's multi_logloss: 0.715592
[2300]	valid_0's multi_logloss: 0.715319
[2400]	valid_0's multi_logloss: 0.715238
Early stopping, best iterat

In [ ]:
n_flod = 17

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': n_est,
    'objective': 'multiclass',
    'random_state': SEED,
    'learning_rate': 0.01,
    'reg_alpha': 1.56e-05,
    'reg_lambda': 0.03167,
    'num_leaves': 300,
    'num_class': n_class,
    'max_depth': 99,
    'max_bin': 450,
    'subsample':0.8,
    'subsamle_freq':3,
    'min_child_samples':4,
    'colsample_bytree':0.39,
    'min_data_in_leaf':17,
    'bagging_fraction':0.9,
    'bagging_freq':8
}

lgb_pred = np.zeros([len(test),3])
train_proba = np.zeros((train_x.shape[0], n_class), dtype=float)

kfold = KFold(n_splits=n_fold, random_state=SEED, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_x)):
  print(f"===== {fold} time =====")

  X_train, X_val = train_x.iloc[train_idx, :], train_x.iloc[val_idx, :]
  y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

  clf = LGBMClassifier(**lgb_params)
  clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=100)

  lgb_pred += clf.predict_proba(test_x)/ n_fold
  train_proba[val_idx] = clf.predict_proba(X_val)

train_prob_dict['lgb2'] = train_proba
test_prob_dict['lgb2'] = lgb_pred

===== 0 time =====
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.823789
[200]	valid_0's multi_logloss: 0.789113
[300]	valid_0's multi_logloss: 0.763914
[400]	valid_0's multi_logloss: 0.748587
[500]	valid_0's multi_logloss: 0.738014
[600]	valid_0's multi_logloss: 0.730789
[700]	valid_0's multi_logloss: 0.725946
[800]	valid_0's multi_logloss: 0.722901
[900]	valid_0's multi_logloss: 0.720545
[1000]	valid_0's multi_logloss: 0.719795
[1100]	valid_0's multi_logloss: 0.719532
Early stopping, best iteration is:
[1060]	valid_0's multi_logloss: 0.719335
===== 1 time =====
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.790119
[200]	valid_0's multi_logloss: 0.756029
[300]	valid_0's multi_logloss: 0.733096
[400]	valid_0's multi_logloss: 0.718248
[500]	valid_0's multi_logloss: 0.708505
[600]	valid_0's multi_logloss: 0.701606
[700]	valid_0's multi_logloss: 0.696386
[800]	valid_0's multi_logloss: 0.692184
[900]	valid_0's multi_logloss: 0.689592
[1000]	valid_0's multi_logloss: 0.687661
[1100]	valid_0's multi_logloss: 0.68685
[1200]	valid_0's multi_logloss: 0.686683
Early stopping, best iteration is:
[1177]	valid_0's multi_logloss: 0.686247
===== 2 time =====
Training until validation scores don't improve for 100 rounds.


...

[100]	valid_0's multi_logloss: 0.822019
[200]	valid_0's multi_logloss: 0.783111
[300]	valid_0's multi_logloss: 0.755654
[400]	valid_0's multi_logloss: 0.737885
[500]	valid_0's multi_logloss: 0.727216
[600]	valid_0's multi_logloss: 0.719841
[700]	valid_0's multi_logloss: 0.714502
[800]	valid_0's multi_logloss: 0.710511
[900]	valid_0's multi_logloss: 0.707299
[1000]	valid_0's multi_logloss: 0.705913
[1100]	valid_0's multi_logloss: 0.704762
[1200]	valid_0's multi_logloss: 0.704575
Early stopping, best iteration is:
[1184]	valid_0's multi_logloss: 0.704214
===== 18 time =====
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.796233
[200]	valid_0's multi_logloss: 0.756752
[300]	valid_0's multi_logloss: 0.728934
[400]	valid_0's multi_logloss: 0.709437
[500]	valid_0's multi_logloss: 0.696779
[600]	valid_0's multi_logloss: 0.688069
[700]	valid_0's multi_logloss: 0.681035
[800]	valid_0's multi_logloss: 0.675453
[900]	valid_0's multi_logloss: 0.671344
[1000]	valid_0's multi_logloss: 0.668248
[1100]	valid_0's multi_logloss: 0.66607
[1200]	valid_0's multi_logloss: 0.663879
[1300]	valid_0's multi_logloss: 0.662792
[1400]	valid_0's multi_logloss: 0.661844
[1500]	valid_0's multi_logloss: 0.662316
Early stopping, best iteration is:
[1423]	valid_0's multi_logloss: 0.661762
===== 19 time =====
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.826249
[200]	valid_0's multi_logloss: 0.789732
[300]	valid_0's multi_logloss: 0.76426
[400]	valid_0's multi_logloss: 0.747896
[500]	valid_0's multi_logloss: 0.738407
[600]	valid_0's multi_logloss: 0.732433
[700]	valid_0's multi_logloss: 0.727678
[800]	valid_0's multi_logloss: 0.724413
[900]	valid_0's multi_logloss: 0.722971
[1000]	valid_0's multi_logloss: 0.721717
[1100]	valid_0's multi_logloss: 0.721737
Early stopping, best iteration is:
[1029]	valid_0's multi_logloss: 0.721651


In [ ]:
from xgboost import XGBClassifier

n_fold = 15

xgb_pred = np.zeros([len(test),3])
train_proba = np.zeros((train_x.shape[0], n_class), dtype=float)

xgb_params={
    'max_depth':7,
    'n_estimators':200,
    'max_features':0.4,
    'learning_rate' : 0.2,
    'min_samples_split': 16,
    'min_samples_leaf': 3,
}

kfold = KFold(n_splits=n_fold, random_state=SEED, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
  print(f"===== {fold} time =====")

  X_train, X_val = train_x.iloc[train_idx, :], train_x.iloc[val_idx, :]
  y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

  clf = XGBClassifier(**xgb_params)
  xgb_clf = clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100)

  xgb_pred += clf.predict_proba(test_x)/ n_fold
  train_proba[val_idx] = clf.predict_proba(X_val)

train_prob_dict['xgb'] = train_proba
test_prob_dict['xgb'] = xgb_pred

===== 0 time =====


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-merror:0.312358
Will train until validation_0-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.312925
[2]	validation_0-merror:0.311791
[3]	validation_0-merror:0.311791
[4]	validation_0-merror:0.310091
[5]	validation_0-merror:0.309524
[6]	validation_0-merror:0.310658
[7]	validation_0-merror:0.310658
[8]	validation_0-merror:0.310658
[9]	validation_0-merror:0.311791
[10]	validation_0-merror:0.311791
[11]	validation_0-merror:0.310658
[12]	validation_0-merror:0.310658
[13]	validation_0-merror:0.311224
[14]	validation_0-merror:0.310658
[15]	validation_0-merror:0.309524
[16]	validation_0-merror:0.310091
[17]	validation_0-merror:0.310658
[18]	validation_0-merror:0.310091
[19]	validation_0-merror:0.309524
[20]	validation_0-merror:0.310091
[21]	validation_0-merror:0.310091
[22]	validation_0-merror:0.309524
[23]	validation_0-merror:0.308957
[24]	validation_0-merror:0.309524
[25]	validation_0-merror:0.308957
[26]	validation_0-merror:0.309524
[27]	validation_0-merror

In [ ]:
import lightgbm as lgb

for fold in range(5):
  lgb.plot_importance(models[fold], figsize=(15, 15))

#Stacking

In [ ]:
Stack_final_X_train = np.concatenate([x for _, x in train_prob_dict.items()], axis=1)
Stack_final_X_test = np.concatenate([x for _, x in test_prob_dict.items()], axis=1)

(26457, 9) (10000, 9)


array([[0.03321658, 0.08471705, 0.88206637, ..., 0.06079659, 0.12352024,
        0.81568316],
       [0.25143838, 0.28720463, 0.46135699, ..., 0.17939956, 0.17628433,
        0.64431609],
       [0.03169293, 0.0880719 , 0.88023517, ..., 0.06575175, 0.12787136,
        0.80637689],
       ...,
       [0.03014336, 0.0891863 , 0.88067034, ..., 0.01508624, 0.04383355,
        0.94108022],
       [0.11457289, 0.23480021, 0.6506269 , ..., 0.0747144 , 0.20240423,
        0.72288137],
       [0.07412246, 0.21783604, 0.7080415 , ..., 0.05892242, 0.20229625,
        0.73878134]])

In [ ]:
Stack_final_X_train = pd.DataFrame(Stack_final_X_train)
Stack_final_X_test = pd.DataFrame(Stack_final_X_test)
Stack_final_X_test.shape

(10000, 9)

In [ ]:
n_fold = 7;

final_pred = np.zeros([len(test),3])

kfold = KFold(n_splits=n_fold, random_state=SEED, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kfold.split(Stack_final_X_train, train_y)):
  print(f" ===== {fold} ===== ")

  X_train, X_val = Stack_final_X_train.iloc[train_idx, :], Stack_final_X_train.iloc[val_idx, :]
  y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

  final = LGBMClassifier(**lgb_params)
  final.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=100)

  final_pred += final.predict_proba(Stack_final_X_test)/n_fold
  # train_pred = final.predict_proba(train_y)
  # print(f"log_loss: {log_loss(to_categorical(y_val['credit']), train_pred)}")

test_prob_dict['final'] = final_pred

 ===== 0 ===== 
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.783592
[200]	valid_0's multi_logloss: 0.74146
[300]	valid_0's multi_logloss: 0.721414
[400]	valid_0's multi_logloss: 0.711408
[500]	valid_0's multi_logloss: 0.706463
[600]	valid_0's multi_logloss: 0.704009
[700]	valid_0's multi_logloss: 0.702973
[800]	valid_0's multi_logloss: 0.702666
[900]	valid_0's multi_logloss: 0.702994
Early stopping, best iteration is:
[808]	valid_0's multi_logloss: 0.702634
 ===== 1 ===== 
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.792904
[200]	valid_0's multi_logloss: 0.746863
[300]	valid_0's multi_logloss: 0.724265
[400]	valid_0's multi_logloss: 0.713084
[500]	valid_0's multi_logloss: 0.707435
[600]	valid_0's multi_logloss: 0.704902
[700]	valid_0's multi_logloss: 0.703783
[800]	valid_0's multi_logloss: 0.703726
Early stopping, best iteration is:
[733]	valid_0's multi_logloss: 0.703486
 ===== 2 ===== 
Training until validation scores don't improve for 100 rounds.


...

[100]	valid_0's multi_logloss: 0.785166
[200]	valid_0's multi_logloss: 0.740311
[300]	valid_0's multi_logloss: 0.71881
[400]	valid_0's multi_logloss: 0.708648
[500]	valid_0's multi_logloss: 0.703603
[600]	valid_0's multi_logloss: 0.701737
[700]	valid_0's multi_logloss: 0.700985
[800]	valid_0's multi_logloss: 0.701199
Early stopping, best iteration is:
[720]	valid_0's multi_logloss: 0.700867
 ===== 5 ===== 
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.771145
[200]	valid_0's multi_logloss: 0.727111
[300]	valid_0's multi_logloss: 0.705548
[400]	valid_0's multi_logloss: 0.694766
[500]	valid_0's multi_logloss: 0.689332
[600]	valid_0's multi_logloss: 0.68664
[700]	valid_0's multi_logloss: 0.685358
[800]	valid_0's multi_logloss: 0.68501
[900]	valid_0's multi_logloss: 0.685135
Early stopping, best iteration is:
[805]	valid_0's multi_logloss: 0.684996
 ===== 6 ===== 
Training until validation scores don't improve for 100 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[100]	valid_0's multi_logloss: 0.778407
[200]	valid_0's multi_logloss: 0.733184
[300]	valid_0's multi_logloss: 0.711402
[400]	valid_0's multi_logloss: 0.700364
[500]	valid_0's multi_logloss: 0.695058
[600]	valid_0's multi_logloss: 0.692672
[700]	valid_0's multi_logloss: 0.691857
[800]	valid_0's multi_logloss: 0.691932
Early stopping, best iteration is:
[754]	valid_0's multi_logloss: 0.691698


# Evaluation

In [ ]:
final_pred

array([[0.04981696, 0.09557595, 0.8546071 ],
       [0.27787076, 0.23115724, 0.490972  ],
       [0.04704505, 0.10610447, 0.84685048],
       ...,
       [0.03871819, 0.11663317, 0.84464864],
       [0.080758  , 0.27884537, 0.64039663],
       [0.08398622, 0.20555853, 0.71045525]])

In [ ]:
submission.loc[:,1:] = test_prob_dict['final']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  """Entry point for launching an IPython kernel.


In [ ]:
submission

,index,0,1,2
0,26457,0.049817,0.095576,0.854607
1,26458,0.277871,0.231157,0.490972
2,26459,0.047045,0.106104,0.846850
3,26460,0.092579,0.135807,0.771614
4,26461,0.052223,0.135060,0.812717
...,...,...,...,...
9995,36452,0.074346,0.259753,0.665901
9996,36453,0.221341,0.307585,0.471074
9997,36454,0.038718,0.116633,0.844649
9998,36455,0.080758,0.278845,0.640397


In [ ]:
submission.to_csv('stacking_3.csv',index=False)